# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/argu/6.1.1/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/CommonFSharp.fs
#!import ../nbs/Async.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let inline testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 120
|> _equal (Some false)

00:00:00 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let inline testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 120 65535
|> Async.RunSynchronously
|> _equal false

00:00:00 #2 [Debug] runWithTimeoutAsync / timeout: 120
False


## waitForPortAccess

In [ ]:
let inline waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 2 else 2)
            (if Runtime.isWindows () then 5 else 1500)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 80 else 80)
            (if Runtime.isWindows () then 150 else 600)

        true
    | _ -> false
|> _equal true

00:00:01 #3 [Debug] 1
00:00:01 #4 [Debug] _1
00:00:01 #5 [Debug] 2
00:00:01 #6 [Debug] 3
00:00:03 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:03 #8 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:05 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:06 #10 [Debug] _2
00:00:06 #11 [Debug] _3
00:00:06 #12 [Debug] 4
00:00:06 #13 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:07 #14 [Warn] waitForPortAccess / port: 5000 / retry: 100
00:00:08 #15 [Debug] _4
00:00:08 #16 [Debug] _5
00:00:10 #17 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:10 #18 [Debug] 5
00:00:10 #19 [Debug] 6
2
2
2
119
119
119
True


In [ ]:
//// test

let port = 5000

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 4 else 2)
            (if Runtime.isWindows () then 15 else 150)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 5 else 0)
            (if Runtime.isWindows () then 20 else 60)

        true
    | _ -> false
|> _equal true

00:00:10 #20 [Debug] 1
00:00:10 #21 [Debug] 2
00:00:10 #22 [Debug] _1
00:00:10 #23 [Debug] 3
00:00:10 #24 [Debug] runWithTimeoutAsync / timeout: 60
00:00:10 #25 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:10 #26 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #27 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #28 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #29 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #30 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #31 [Debug] _2
00:00:11 #32 [Debug] _3
00:00:11 #33 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #34 [Debug] 4
00:00:11 #35 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:11 #36 [Debug] _4
00:00:11 #37 [Debug] _5
00:00:11 #38 [Debug] runWithTimeoutAsync / timeout: 60
00:00:11 #39 [Debug] 5
00:00:11 #40 [Debug] 6
7
7
7
8
8
8
True


## getAvailablePort

In [ ]:
let inline getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 180
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 1200)

        true
    | _ -> false
|> _equal true

00:00:12 #41 [Debug] 1
00:00:12 #42 [Debug] 2
00:00:12 #43 [Debug] _1
00:00:12 #44 [Debug] _2
00:00:12 #45 [Debug] _3
00:00:12 #46 [Debug] 3
00:00:14 #47 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:14 #48 [Debug] 4
00:00:14 #49 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:16 #50 [Warn] waitForPortAccess / port: 5000 / retry: 100
00:00:16 #51 [Debug] _4
00:00:16 #52 [Debug] _5
00:00:18 #53 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
00:00:18 #54 [Debug] 5
00:00:18 #55 [Debug] 6
5002
109
109
109
True


In [ ]:
//// test

let port = 5000

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 60
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 10 else 1)
            (if Runtime.isWindows () then 30 else 120)

        true
    | _ -> false
|> _equal true

00:00:18 #56 [Debug] 1
00:00:18 #57 [Debug] 2
00:00:18 #58 [Debug] _1
00:00:18 #59 [Debug] _2
00:00:18 #60 [Debug] _3
00:00:18 #61 [Debug] 3
00:00:18 #62 [Debug] runWithTimeoutAsync / timeout: 60
00:00:19 #63 [Debug] 4
00:00:19 #64 [Warn] waitForPortAccess / port: 5000 / retry: 0
00:00:19 #65 [Debug] _4
00:00:19 #66 [Debug] _5
00:00:19 #67 [Debug] runWithTimeoutAsync / timeout: 60
00:00:19 #68 [Debug] 5
00:00:19 #69 [Debug] 6
5002
17
17
17
True
